# <font color='brown'> **Backtesting (código clase)**
## <font color='rosybrown'> **Arantza Gomez Haro Gamboa**

### <font color='rosybrown'> **Supuestos:**

- Trades @ close
- Comisión = 0.125%
- SL = TP = 5%
- Cash = $1M
- N Shares = 50
- No taxes
- No leverages
- Margin Acc = 50%
- Borrow rate = 0.25%
- Timeframe = 5 minutes


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ta

sns.set_theme()

In [2]:
data = pd.read_csv('aapl_5m_train.csv').dropna()
data.head()

,Unnamed: 0,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume
0,0,1609770600,0,2021-01-04 14:30:00,133.570007,133.611602,132.389999,132.809997,6624663.0
1,1,1609770900,0,2021-01-04 14:35:00,132.750000,132.750000,131.809997,131.889999,2541553.0
2,2,1609771200,0,2021-01-04 14:40:00,131.500000,132.339996,131.500000,132.059997,2492415.0
3,3,1609771500,0,2021-01-04 14:45:00,132.000000,132.250000,131.899993,132.250000,1859131.0
4,4,1609771800,0,2021-01-04 14:50:00,132.000000,132.018096,131.520004,131.589996,1780105.0


In [3]:
rsi_indicator = ta.momentum.RSIIndicator(data.Close, window=20)
data['RSI'] = rsi_indicator.rsi()
data.head()

,Unnamed: 0,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume,RSI
0,0,1609770600,0,2021-01-04 14:30:00,133.570007,133.611602,132.389999,132.809997,6624663.0,NaN
1,1,1609770900,0,2021-01-04 14:35:00,132.750000,132.750000,131.809997,131.889999,2541553.0,NaN
2,2,1609771200,0,2021-01-04 14:40:00,131.500000,132.339996,131.500000,132.059997,2492415.0,NaN
3,3,1609771500,0,2021-01-04 14:45:00,132.000000,132.250000,131.899993,132.250000,1859131.0,NaN
4,4,1609771800,0,2021-01-04 14:50:00,132.000000,132.018096,131.520004,131.589996,1780105.0,NaN


In [4]:
historic = data.dropna()
historic['buy_signal'] = historic.RSI < 20
historic['sell_signal'] = historic.RSI > 80

C:\Users\arant\AppData\Local\Temp\ipykernel_5008\3205292575.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historic['buy_signal'] = historic.RSI < 20
C:\Users\arant\AppData\Local\Temp\ipykernel_5008\3205292575.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historic['sell_signal'] = historic.RSI > 80


In [5]:
historic.head()

,Unnamed: 0,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume,RSI,buy_signal,sell_signal
19,19,1609776300,0,2021-01-04 16:05:00,130.309997,130.549896,129.660003,129.866699,3030766.0,26.683236,False,False
20,20,1609776600,0,2021-01-04 16:10:00,129.860000,130.239898,129.399993,130.089996,2231137.0,30.749593,False,False
21,21,1609776900,0,2021-01-04 16:15:00,130.080093,130.354995,129.904602,130.324996,1328068.0,34.758166,False,False
22,22,1609777200,0,2021-01-04 16:20:00,130.324005,130.419998,129.930099,129.996398,1233893.0,32.029264,False,False
23,23,1609777500,0,2021-01-04 16:25:00,129.994995,130.206695,129.880004,129.895004,1144809.0,31.232801,False,False


In [6]:
COM = 0.125 / 100
SL = 0.05
TP = 0.05
BORROW_RATE = 0.25 / 100

cash = 1_000_000
n_shares = 50

In [7]:
from dataclasses import dataclass

@dataclass
class Operation:
    time: str
    price: float
    stop_loss: float
    take_profit: float
    n_shares: int
    type: str

In [8]:
active_positions: list[Operation] = []

for i, row in historic.iterrows():
    # close positions
    for position in active_positions.copy():

        if row.Close > position.take_profit or row.Close < position.stop_loss:
            # close position

            cash += row.Close * position.n_shares * (1 - COM)
            active_positions.remove(position)

    # check signal

    if not row.buy_signal:
        continue
    
    #check if you have enough cash

    if cash < row.Close * n_shares * (1 + COM):
        continue

    #discount the cost

    cash -= row.Close * n_shares * (1 + COM)
    # save the operation as active position

    active_positions.append(
        Operation(
            time=row.Datetime,
            price=row.Close,
            stop_loss=row.Close * (1 - SL),
            take_profit=row.Close * (1 + TP),
            n_shares=n_shares,
            type='LONG'
        )
    )

In [9]:
cash

1007816.8160896875

In [10]:
active_positions

[]